In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [2]:
import sys

In [3]:
sys.path.append(os.path.abspath('..'))

# 1. Translate the English training set into Russian

In [5]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

In [6]:
detox_en_train = pd.read_csv('../data/english_data/en_data.csv', sep='\t')

In [7]:
pd.options.display.max_colwidth = 200

In [8]:
detox_en_train.shape

(19766, 9)

In [9]:
detox_en_train.sample(5)

,idx,toxic_comment,neutral_comment,toxicity_score,dataset,toxic,confidence_toxic,is_match,confidence_is_match
14308,162672,"read the headline and my only reaction was , "" fuck , again ? ? ? ? """,Read the headline and my only reaction was. Again???,0.997740,reddit,False,0.9987,True,0.9679
7850,41551,that shit irritates me with a passion lol,That irritate me,0.998720,twitter,False,0.9972,True,0.9777
8728,21636,put alot of that green shit on,put alot of that green stuff on,0.998061,twitter,False,0.9987,True,0.9906
3410,44090,those cunts need sorting out !,Those guys need to sort out.,0.997767,jigsaw,False,0.9994,True,0.9230
10080,44179,i suppose it 's more fun to make shit up,i suppose it 's more fun to make things up.,0.986727,twitter,False,0.9914,True,0.9924


In [10]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /home/dale/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
import re

def split_by_symbol(text, symbol=',', max_len=400):
    if len(text) <= max_len:
        return [text]
    chunks = re.split(symbol, text)
    if len(chunks) <= 1:
        return [text]
    result = [chunks[0]]
    for chunk in chunks[1:]:
        result.append(symbol)
        result.append(chunk)
    return result

def join_texts(texts, max_len=400):
    result = []
    prev_text = ''
    for text in texts:
        if len(text) + len(prev_text) > max_len:
            result.append(prev_text)
            prev_text = text
        else:
            prev_text = prev_text + text
    result.append(prev_text)
    return result

def hard_split(text, max_len=300):
    parts = list(sent_tokenize(text))
    result = []
    for part in parts:
        chunks = [part]
        for symbol in [',', '-', ' ']:
            chunks = [c2 for c in chunks for c2 in split_by_symbol(c, symbol, max_len=max_len)]
        result.extend(chunks)
    result = join_texts(result, max_len=max_len)
    return result

In [12]:
'''
How to obtain a fresh SID:
* go to translate.yandex.ru
* open the "network" panel of the developers console
* enter any text in the translation form
* find the request to "https://translate.yandex.net/api/v1/tr.json/translate" and copy its first parameter ("id")
'''

import requests

SID = '7d9e19fd.62960895.db4a787d.74722d74657874-8-0'

def translate_yandex(search_str, direction='en-ru', full_response=False):
    try:
        url = f'https://translate.yandex.net/api/v1/tr.json/translate?id={SID}&srv=tr-text&lang={direction}&reason=auto&format=text'

        post_header = {}
        post_header['Accept'] = '*/*'
        post_header['Accept-Encoding'] = 'gzip, deflate'
        post_header['Accept-Language'] = 'en-US,en;q=0.9'
        post_header['Cache-Control'] = 'no-cache'
        post_header['Connection'] = 'keep-alive'
        post_header['Content-Type'] = 'application/x-www-form-urlencoded'
        post_header['Host'] = 'translate.yandex.com'
        post_header['Referer'] = 'https://translate.yandex.com/'
        post_header['User-Agent'] = 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 YaBrowser/21.8.2.383 Yowser/2.5 Safari/537.36'


        data_payload = {'text': search_str, 'options': '4'}

        resp = requests.post(url, headers=post_header, data=data_payload).json()
        #print(resp)
        if full_response:
            return resp

        if resp.get('message') == 'The text size exceeds the maximum':
            parts = hard_split(search_str)
            if len(parts) > 1:
                return 200, ' '.join([translate_yandex(part, dir=dir)[1] for part in parts])

        return resp['code'], resp['text'][0]
    except Exception as e:
        raise e
        return 0, ''

In [14]:
translate_yandex('go to hell')

(200, 'иди к черту')

In [15]:
detox_en_train.toxic_comment[:5].tolist()

['he had steel balls too !',
 'dude should have been taken to api , he would be right at home with all the other knuckleheads there',
 'im not gonna sell the fucking picture , i just want to contribute to the fucking article .',
 'the garbage that is being created by cnn and other news agencies is outrageous .',
 'the reason they dont exist is because neither is a pathological liar like trump .']

In [17]:
[translate_yandex(t)[1] for t in detox_en_train.toxic_comment[:5]]

['у него тоже были стальные яйца!',
 'чувака надо было отвезти в апи, он был бы там как дома со всеми остальными болванами',
 'я не собираюсь продавать эту гребаную фотографию, я просто хочу внести свой вклад в гребаную статью.',
 'мусор, который создают cnn и другие информационные агентства, возмутителен.',
 'причина, по которой их не существует, заключается в том, что ни один из них не является таким патологическим лжецом, как Трамп.']

In [18]:
[translate_yandex(t)[1] for t in detox_en_train.neutral_comment[:5]]

['он тоже был храбрым!',
 'Было бы хорошо, если бы он пошел в апи. Он бы вписался.',
 'Я не собираюсь продавать фотографию, я просто хочу внести свой вклад в статью.',
 'новости, которые создают cnn и другие информационные агентства, возмутительны.',
 'Причина, по которой их не существует, заключается в том, что ни один из них не лжет так, как Трамп']

In [ ]:
toxic_ru = []
neutral_ru = []
for i in trange(0, len(detox_en_train)):
    toxic_ru.append(translate_yandex(detox_en_train.toxic_comment[i])[1])
    neutral_ru.append(translate_yandex(detox_en_train.neutral_comment[i])[1])

  0%|          | 0/19766 [00:00<?, ?it/s]

In [27]:
detox_en_train['toxic_ru'] = toxic_ru
detox_en_train['neutral_ru'] = neutral_ru

In [28]:
detox_en_train.sample(5)

,idx,toxic_comment,neutral_comment,toxicity_score,dataset,toxic,confidence_toxic,is_match,confidence_is_match,toxic_ru,neutral_ru
15032,119985,are you implying the cops are pussies - - thus they fired out of legitimate fear for their safety ?,"Are you implying the cops are cowards - thus, they fired out of legitimate fear for their own safety?",0.996316,reddit,False,0.9824,True,0.9482,"вы намекаете на то, что копы - слабаки, и поэтому они стреляли из законного страха за свою безопасность?","Вы намекаете на то, что копы трусы - таким образом, они стреляли из законного страха за свою собственную безопасность?"
14607,151094,"yea something like "" surgeon general said that an apple a day or some shit","yea something like "" surgeon general said that an apple a day or something",0.994066,reddit,False,0.9955,True,0.9757,"да, что-то вроде ""главный хирург сказал, что яблоко в день или что-то в этом роде","да, что-то вроде ""главный хирург сказал, что яблоко в день или что-то в этом роде"
3781,77029,the only logical conclusion i see here is that trump is a lying sack .,The only logical conclusion I can draw from this is that Trump is a lied,0.997076,jigsaw,False,0.9564,True,0.9834,"единственный логический вывод, который я вижу здесь, заключается в том, что Трамп - лживый мешок.","Единственный логический вывод, который я могу сделать из этого, заключается в том, что Трамп - лжец"
1002,35987,what kind of rat hole do these monsters come from ?,what kind of a hole these came from,0.993900,jigsaw,False,0.9990,True,0.9451,из какой крысиной норы появляются эти монстры?,из какой дыры они взялись
6652,3542,man this shit is gettin wiild over here lol .,man this is gettin wiild over here lol .,0.996682,twitter,False,0.9957,True,0.9924,"чувак, это дерьмо становится все более распространенным, лол.","блин, это уже начинает надоедать, лол."


In [37]:
detox_en_train.to_csv('detox_en2ru_yandex.tsv', index=None, sep='\t')

# Start from here

In [39]:
(detox_en_train.toxic_ru == detox_en_train.neutral_ru).mean()

0.011079631690782151

In [40]:
detox_en_train.toxic_ru.str.len().describe()

count    19766.000000
mean        60.839269
std         24.365534
min          8.000000
25%         41.000000
50%         57.000000
75%         77.000000
max        173.000000
Name: toxic_ru, dtype: float64

In [41]:
detox_en_train.neutral_ru.str.len().describe()

count    19766.000000
mean        51.011889
std         24.548076
min          1.000000
25%         32.000000
50%         47.000000
75%         67.000000
max        165.000000
Name: neutral_ru, dtype: float64

In [42]:
from textdistance import levenshtein

In [43]:
detox_en_train['edit_distance_en'] = [levenshtein.distance(*row) for row in detox_en_train[['toxic_comment', 'neutral_comment']].values]
detox_en_train['edit_distance_ru'] = [levenshtein.distance(*row) for row in detox_en_train[['toxic_ru', 'neutral_ru']].values]

detox_en_train['edit_sim_en'] = [levenshtein.normalized_similarity(*row) for row in detox_en_train[['toxic_comment', 'neutral_comment']].values]
detox_en_train['edit_sim_ru'] = [levenshtein.normalized_similarity(*row) for row in detox_en_train[['toxic_ru', 'neutral_ru']].values]

In [44]:
detox_en_train.describe()

,idx,toxicity_score,confidence_toxic,confidence_is_match,edit_distance_en,edit_distance_ru,edit_sim_en,edit_sim_ru
count,19766.000000,19766.000000,19766.000000,19766.000000,19766.000000,19766.000000,19766.000000,19766.000000
mean,78035.783416,0.990181,0.984174,0.972585,15.705757,22.234139,0.696492,0.618786
std,60839.351686,0.019532,0.025847,0.025094,10.989535,13.887489,0.186762,0.200999
min,7.000000,0.800983,0.523700,0.800000,0.000000,0.000000,0.000000,0.000000
25%,31839.000000,0.991065,0.981100,0.961100,8.000000,12.000000,0.590909,0.480000
50%,63895.000000,0.996133,0.994100,0.980500,12.000000,19.000000,0.741935,0.649123
75%,101255.250000,0.998179,0.998500,0.991100,20.000000,29.000000,0.838710,0.776316
max,238836.000000,0.999647,1.000000,0.999900,105.000000,129.000000,1.000000,1.000000


In [45]:
detox_en_train[['edit_distance_en', 'edit_sim_en']].quantile([0.01, 0.05, 0.90, 0.99])

,edit_distance_en,edit_sim_en
0.01,3.0,0.192308
0.05,5.0,0.312500
0.90,30.0,0.900000
0.99,56.0,0.961538


In [46]:
# detox_en_train[['edit_distance_en', 'edit_sim_en']].quantile([0.01, 0.05, 0.90, 0.99])
print((detox_en_train.edit_distance_ru < 3).mean())
print((detox_en_train.edit_distance_ru > 56).mean())

print((detox_en_train.edit_sim_ru < 0.19).mean())
print((detox_en_train.edit_sim_ru > 0.96).mean())

0.017150662754224427
0.027774967115248406
0.021552160275220073
0.015987048467064658


# 2. Train the Russian model 

In [4]:
import pandas as pd

In [47]:
detox_en2ru = pd.read_csv('detox_en2ru_yandex.tsv', sep='\t')

In [48]:
from datasets import Dataset, DatasetDict

In [49]:
from sklearn.model_selection import train_test_split

Filter text pairs by similarity to escape translation artifacts

In [50]:
detox_en2ru_filtered = detox_en2ru[
    (detox_en2ru.edit_distance_ru >= detox_en2ru.edit_distance_en.quantile(0.01)) 
    & (detox_en2ru.edit_distance_ru <= detox_en2ru.edit_distance_en.quantile(0.99)) 
    & (detox_en2ru.edit_sim_ru >= detox_en2ru.edit_sim_en.quantile(0.01)) 
    & (detox_en2ru.edit_sim_ru <= detox_en2ru.edit_sim_en.quantile(0.99))
]

print(detox_en2ru.shape)
print(detox_en2ru_filtered.shape)

(19766, 15)
(18488, 15)


In [51]:
train, val = train_test_split(detox_en2ru_filtered, random_state=1, test_size=500)

In [53]:
raw_data = DatasetDict({
    'train': Dataset.from_dict({'text': train.toxic_ru, 'target': train.neutral_ru}),
    'dev': Dataset.from_dict({'text': val.toxic_ru, 'target': val.neutral_ru}),
})
raw_data

DatasetDict({
    train: Dataset({
        features: ['text', 'target'],
        num_rows: 17988
    })
    dev: Dataset({
        features: ['text', 'target'],
        num_rows: 500
    })
})

### Compute and evaluate baselines

In [54]:
from sacrebleu import CHRF
chrfpp = CHRF(word_order=2)

Baseline chrf++: 60% if not change the texts. 

In [55]:
chrfpp.corpus_score(val.toxic_ru.tolist(), [val.neutral_ru.tolist()]).score

66.09829950381689

A baseline that removes bad words

In [56]:
from nltk import wordpunct_tokenize
from collections import Counter

def detokenize(text):
    for symbol in ",.?!'":
        text = text.replace(' ' + symbol, symbol)
    return text

class Remover:
    def __init__(self, ratio_threshold=2):
        self.ratio_threshold = ratio_threshold
    def fit(self, x, y):
        self.x_count = Counter(w.lower() for text in x for w in wordpunct_tokenize(text))
        self.y_count = Counter(w.lower() for text in y for w in wordpunct_tokenize(text))
    def predict(self, x):
        results = []
        for text in x:
            words = []
            for w in wordpunct_tokenize(text):
                key = w.lower()
                if (self.x_count[key] + 1) / (self.y_count[key] + 1) > self.ratio_threshold:
                    continue
                words.append(w)
            results.append(detokenize(' '.join(words)))
        return results

In [57]:
remover = Remover(2.0)
remover.fit(train.toxic_ru, train.neutral_ru)

In [58]:
chrfpp.corpus_score(remover.predict(val.toxic_ru), [val.neutral_ru.tolist()]).score

66.00557993346862

A simple word-based translation

In [59]:
from collections import deque
from itertools import product
from tqdm.auto import tqdm

# https://johnlekberg.com/blog/2020-10-25-seq-align.html


def needleman_wunsch(x, y, sim=None, verbose=False):
    """Run the Needleman-Wunsch algorithm on two sequences.

    x, y -- sequences.

    Code based on pseudocode in Section 3 of:

    Naveed, Tahir; Siddiqui, Imitaz Saeed; Ahmed, Shaftab.
    "Parallel Needleman-Wunsch Algorithm for Grid." n.d.
    https://upload.wikimedia.org/wikipedia/en/c/c4/ParallelNeedlemanAlgorithm.pdf
    """
    N, M = len(x), len(y)
    if sim is None:
        s = lambda a, b: int(a == b)
    else:
        s = sim

    DIAG = -1, -1
    LEFT = -1, 0
    UP = 0, -1

    # Create tables F and Ptr
    F = {}
    Ptr = {}

    F[-1, -1] = 0
    for i in range(N):
        F[i, -1] = -i
    for j in range(M):
        F[-1, j] = -j

    option_Ptr = DIAG, LEFT, UP
    for i, j in product(range(N), range(M)):
        option_F = (
            F[i - 1, j - 1] + s(x[i], y[j]),
            F[i - 1, j] - 1,
            F[i, j - 1] - 1,
        )
        F[i, j], Ptr[i, j] = max(zip(option_F, option_Ptr))

    # Work backwards from (N - 1, M - 1) to (0, 0)
    # to find the best alignment.
    alignment = deque()
    i, j = N - 1, M - 1
    if verbose:
        tq = tqdm(total=max(N, M))
        
    while i >= 0 and j >= 0:
        direction = Ptr[i, j]
        if direction == DIAG:
            element = i, j
        elif direction == LEFT:
            element = i, None
        elif direction == UP:
            element = None, j
        alignment.appendleft(element)
        di, dj = direction
        i, j = i + di, j + dj
    while i >= 0:
        alignment.appendleft((i, None))
        i -= 1
    while j >= 0:
        alignment.appendleft((None, j))
        j -= 1

    return list(alignment)

In [60]:
from collections import Counter, defaultdict
import nltk
from tqdm.auto import tqdm, trange
import numpy as np


class TextReplacer:
    def __init__(self, max_n=3, smooth_n=10, min_n=10, min_p=0.01):
        self.max_n = max_n
        self.smooth_n = smooth_n
        self.min_n = min_n
        self.min_p = min_p
        
        self.replace_proba = {}
        self.replaced_tuples = {}
        
    def tokenize(self, text):
        return nltk.wordpunct_tokenize('_bos_ ' + text + ' _eos_')
    
    def detokenize(self, text):
        text = text.strip()
        for symbol in '.,?!':
            text = text.replace(' ' + symbol, symbol)
        if text.startswith('_bos_'):
            text = text[5:]
        if text.endswith('_eos_'):
            text = text[:-5]
        return text.strip()
    
    def fit(self, x_train, y_train):
        raw_counts = Counter()
        replace_counts = Counter()
        
        for i in trange(len(x_train)):
            xx, yy = x_train[i], y_train[i]
            xx, yy = self.tokenize(xx), self.tokenize(yy)
            alignment = needleman_wunsch(xx, yy)
            ixx, iyy = list(zip(*alignment))
            for gram_size in range(1, self.max_n + 1):
                for start in range(len(ixx) - gram_size + 1):
                    xgram = tuple([xx[c] for c in ixx[start: start + gram_size] if c is not None])
                    ygram = tuple([yy[c] for c in iyy[start: start + gram_size] if c is not None])
                    if xgram:
                        xg, yg = ' '.join([''] + list(xgram) + ['']), ' '.join([''] + list(ygram) + [''])
                        raw_counts[xg] += 1
                        if xgram != ygram:
                            replace_counts[(xg, yg)] += 1
    
        self.replace_proba = defaultdict(list)
        self.replaced_tuples = dict()

        for pair, n_sub in replace_counts.most_common():
            if n_sub >= self.min_n:
                xx, yy = pair
                pr = n_sub / (self.smooth_n + raw_counts[xx])
                if pr >= self.min_p:
                    self.replace_proba[xx].append([yy, pr])
                    self.replaced_tuples[tuple(xx.strip().split())] = raw_counts[xx]

        for k, v in self.replace_proba.items():
            tot = sum(p for r, p in v)
            if tot < 1:
                v.append([k, 1 - tot])
        
        return self

    def transform_one(self, text, n_out=None, temperature=None):
        xx = self.tokenize(text)
        found_grams = []
        for gram_size in range(1, self.max_n + 1):
            for start in range(len(xx) - gram_size + 1):
                xgram = tuple([c for c in xx[start: start + gram_size] if c is not None])
                if xgram and xgram in self.replaced_tuples:
                    found_grams.append((xgram, self.replaced_tuples[xgram], len(xgram)))
        found_grams = sorted(found_grams, key=lambda x: (x[2], x[1]), reverse=True)
        
        results = []
        for i in range(n_out or 1):
            untext = ' '.join([''] + xx + [''])
            for gram, gn, gl in found_grams:
                gram_text = ' '.join([''] + list(gram) + [''])
                reps, ww = zip(*self.replace_proba[gram_text])
                if not temperature:
                    chosen_rep = list(reps)[np.argmax(ww)]
                else: # chose randomly
                    weights = [w ** (1 / temperature) for w in ww]
                    chosen_rep = random.choices(list(reps), weights=weights)[0]
                untext = untext.replace(gram_text, chosen_rep)
            results.append(self.detokenize(untext))
        if not n_out:
            return results[0]
        return results
    
    def transform(self, texts, n_out=None, temperature=None):
        return [self.transform_one(text, n_out=n_out, temperature=temperature) for text in tqdm(texts)]

In [61]:
replacer = TextReplacer(min_n=3)
replacer.fit(train.toxic_ru.tolist(), train.neutral_ru.tolist())

  0%|          | 0/17988 [00:00<?, ?it/s]

In [62]:
replacer.transform_one("Какого хуя тут делает этот придурок?")

'Какого хуя тут делает этот?'

In [63]:
chrfpp.corpus_score(replacer.transform(val.toxic_ru), [val.neutral_ru.tolist()]).score

  0%|          | 0/500 [00:00<?, ?it/s]

67.35076823754419

### Prepare test methods

In [64]:
def paraphrase(
    text, model, tokenizer, n=None, max_length="auto", beams=5,
):
    texts = [text] if isinstance(text, str) else text
    inputs = tokenizer(texts, return_tensors="pt", padding=True)["input_ids"].to(
        model.device
    )

    if max_length == "auto":
        max_length = inputs.shape[1] + 10

    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=10.0,
        max_length=max_length,
        min_length=int(0.5 * max_length),
        num_beams=beams,
        #forced_bos_token_id=tokenizer.lang_code_to_id[tokenizer.tgt_lang],
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]

    if not n and isinstance(text, str):
        return texts[0]
    return texts[0]

In [65]:
test_data = pd.read_csv('../data/russian_data/test.tsv', sep='\t')
test_inputs = test_data["toxic_comment"].values.tolist()

## Fine-tune mBART

In [66]:
base_model = 'facebook/mbart-large-50'

In [67]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [68]:
model = AutoModelForSeq2SeqLM.from_pretrained(base_model)# .cuda()
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [69]:
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, padding=True)
    labels = tokenizer(examples["target"], padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [70]:
tok_data = raw_data.map(preprocess_function, batched=True)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [71]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [72]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/home/dale/models/detox-parallel/translate-en2ru_yandex-full-mbart",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1, # 8 is too much 
    weight_decay=1e-5,
    max_steps=10_000,
    learning_rate=1e-5,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=1,
    eval_steps=500, 
    save_steps=500,
    logging_steps=500,
    load_best_model_at_end=True,
    # trying to save memory: see https://huggingface.co/docs/transformers/performance
    fp16=True,
    gradient_checkpointing=True,
    optim="adafactor",
    gradient_accumulation_steps=1,
)

In [73]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tok_data["train"],
    eval_dataset=tok_data["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [74]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: target, text. If target, text are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 17988
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10000
/home/dale/p3/lib/python3.7/site-packages/transformers/trainer.py:1599: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Step,Training Loss,Validation Loss
500,2.652800,0.436565
1000,0.451400,0.390012
1500,0.408900,0.368482
2000,0.408100,0.357230
2500,0.363600,0.349286
3000,0.315700,0.347586
3500,0.326700,0.339857
4000,0.315200,0.338116
4500,0.315500,0.336197
5000,0.254800,0.340212


The following columns in the evaluation set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: target, text. If target, text are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
Saving model checkpoint to /home/dale/models/detox-parallel/translate-en2ru_yandex-full-mbart/checkpoint-500
Configuration saved in /home/dale/models/detox-parallel/translate-en2ru_yandex-full-mbart/checkpoint-500/config.json
Model weights saved in /home/dale/models/detox-parallel/translate-en2ru_yandex-full-mbart/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /home/dale/models/detox-parallel/translate-en2ru_yandex-full-mbart/checkpoint-500/tokenizer_config.json
Special tokens file saved in /home/dale/models/detox-parallel/translate-en2ru_yandex-full-mbart/checkpoint-500/special_tokens_map.json
The following columns in the evaluatio

TrainOutput(global_step=10000, training_loss=0.404996280670166, metrics={'train_runtime': 3740.1318, 'train_samples_per_second': 21.39, 'train_steps_per_second': 2.674, 'total_flos': 8126355688980480.0, 'train_loss': 0.404996280670166, 'epoch': 4.45})

In [75]:
from tqdm.auto import tqdm, trange

In [76]:
preds = []
model.eval()
for text in tqdm(val.toxic_ru):
    with torch.inference_mode():
        out = tokenizer.decode(
            model.generate(**tokenizer(text, return_tensors='pt').to(model.device), num_beams=5, max_length=256)[0], 
            skip_special_tokens=True,
        )
        preds.append(out)
        
print(chrfpp.corpus_score(preds, [val.neutral_ru.tolist()]).score)

  0%|          | 0/500 [00:00<?, ?it/s]

68.33748671933068


In [77]:
val.toxic_ru[:5]

4576     что делать? пожалуйста, скажи, что ты ни хрена с собой не делаешь.
7327      пошел ты, rt только что обчистил эту задницу на словах с друзьями
7726          полчаса спустя, а он все еще там, "гребаная ленивая задница"!
16723       я никогда не рассчитывал, что правительство спасет мою задницу.
10466                 у нее большая задница, лол, и она встречалась с Канье
Name: toxic_ru, dtype: object

In [78]:
preds[:5]

['что делать? пожалуйста, скажите, что вы ничего не делаете',
 'ты только что обчистил эту задницу на словах с друзьями',
 'полчаса спустя, а он все еще там!',
 'Я никогда не рассчитывал на то, что правительство спасет меня',
 'У нее большие проблемы, лол, и она встречалась с Канье']

In [79]:
test_outputs = [paraphrase(text, model, tokenizer) for text in tqdm(test_inputs)]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [81]:
path = '../results/translate-train_yandex-full-mbart/'  # partial training results; need to train further.
if not os.path.exists(path):
    os.mkdir(path)

In [82]:
with open(path + 'results_ru.txt', 'w') as f:
    for text in test_outputs:
        f.write(text+'\n')

### Fine-tune mBART with BOTH English (original) and Russian (translated) data

In [83]:
raw_data = DatasetDict({
    'train': Dataset.from_dict({
        'text': train.toxic_ru.tolist() + train.toxic_comment.tolist(), 
        'target': train.neutral_ru.tolist() + train.neutral_comment.tolist()}),
    'dev': Dataset.from_dict({'text': val.toxic_ru, 'target': val.neutral_ru}),
})
raw_data

DatasetDict({
    train: Dataset({
        features: ['text', 'target'],
        num_rows: 35976
    })
    dev: Dataset({
        features: ['text', 'target'],
        num_rows: 500
    })
})

In [84]:
base_model = 'facebook/mbart-large-50'

In [85]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(base_model)# .cuda()
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [87]:
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, padding=True)
    labels = tokenizer(examples["target"], padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [88]:
tok_data = raw_data.map(preprocess_function, batched=True)

  0%|          | 0/36 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [89]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [90]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/home/dale/models/detox-parallel/translate-en2ru-full_bilingual-mbart",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1, # 8 is too much 
    weight_decay=1e-5,
    max_steps=10_000,
    learning_rate=1e-5,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=1,
    eval_steps=500, 
    save_steps=500,
    logging_steps=500,
    load_best_model_at_end=True,
    # trying to save memory: see https://huggingface.co/docs/transformers/performance
    fp16=True,
    gradient_checkpointing=True,
    optim="adafactor",
    gradient_accumulation_steps=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [91]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tok_data["train"],
    eval_dataset=tok_data["dev"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [92]:
1

1

In [93]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: target, text. If target, text are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 35976
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10000
/home/dale/p3/lib/python3.7/site-packages/transformers/trainer.py:1599: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Step,Training Loss,Validation Loss
500,2.657300,0.453836
1000,0.450800,0.406475
1500,0.419700,0.381386
2000,0.392700,0.370062
2500,0.394500,0.361941
3000,0.382200,0.359046
3500,0.373500,0.353909
4000,0.369400,0.345452
4500,0.364200,0.341724
5000,0.306900,0.348707


The following columns in the evaluation set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: target, text. If target, text are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
Saving model checkpoint to /home/dale/models/detox-parallel/translate-en2ru-full_bilingual-mbart/checkpoint-500
Configuration saved in /home/dale/models/detox-parallel/translate-en2ru-full_bilingual-mbart/checkpoint-500/config.json
Model weights saved in /home/dale/models/detox-parallel/translate-en2ru-full_bilingual-mbart/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /home/dale/models/detox-parallel/translate-en2ru-full_bilingual-mbart/checkpoint-500/tokenizer_config.json
Special tokens file saved in /home/dale/models/detox-parallel/translate-en2ru-full_bilingual-mbart/checkpoint-500/special_tokens_map.json
Deleting older checkpoi

TrainOutput(global_step=10000, training_loss=0.45296048736572264, metrics={'train_runtime': 3758.2116, 'train_samples_per_second': 21.287, 'train_steps_per_second': 2.661, 'total_flos': 7893770179706880.0, 'train_loss': 0.45296048736572264, 'epoch': 2.22})

In [94]:
from tqdm.auto import tqdm, trange

In [95]:
preds = []
model.eval()
for text in tqdm(val.toxic_ru):
    with torch.inference_mode():
        out = tokenizer.decode(
            model.generate(**tokenizer(text, return_tensors='pt').to(model.device), num_beams=5, max_length=256)[0], 
            skip_special_tokens=True,
        )
        preds.append(out)

  0%|          | 0/500 [00:00<?, ?it/s]

In [96]:
print(chrfpp.corpus_score(preds, [val.neutral_ru.tolist()]).score)

68.56487845972582


In [97]:
val.toxic_ru[:5]

4576     что делать? пожалуйста, скажи, что ты ни хрена с собой не делаешь.
7327      пошел ты, rt только что обчистил эту задницу на словах с друзьями
7726          полчаса спустя, а он все еще там, "гребаная ленивая задница"!
16723       я никогда не рассчитывал, что правительство спасет мою задницу.
10466                 у нее большая задница, лол, и она встречалась с Канье
Name: toxic_ru, dtype: object

In [98]:
preds[:5]

['что делать? пожалуйста, скажите, что ты ничего не делаешь с собой.',
 'Ты только что обчистил его на словах с друзьями',
 'полчаса спустя, а он все еще там!',
 'Я никогда не рассчитывал, что правительство спасет меня.',
 'У нее большие проблемы, лол, и она встречалась с Канье']

In [99]:
test_outputs = [paraphrase(text, model, tokenizer) for text in tqdm(test_inputs)]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [100]:
path = '../results/translate-train_yandex-full_bilingual-mbart/'  # partial training results; need to train further.
if not os.path.exists(path):
    os.mkdir(path)

In [101]:
with open(path + 'results_ru.txt', 'w') as f:
    for text in test_outputs:
        f.write(text+'\n')

# 3. Evaluation results

```
python evaluate_ru.py \
    --result_filename scores \
    --input_dir results/translate-train_yandex-full-mbart \
    --output_dir results
    
Style accuracy:       0.4556241035461426
Meaning preservation: 0.8801424503326416
Joint fluency:        -0.10965493321418762
Joint score:          -0.04347466304898262
Scores after calibration:
Style accuracy:       0.5100616812705994
Meaning preservation: 0.8203034400939941
Joint fluency:        0.8738968372344971
Joint score:          0.35575705766677856
```

```
python evaluate_ru.py \
    --result_filename scores \
    --input_dir results/translate-train_yandex-full_bilingual-mbart \
    --output_dir results
    
Style accuracy:       0.5160583257675171
Meaning preservation: 0.9036513566970825
Joint fluency:        -0.07349134981632233
Joint score:          -0.03711703419685364
Scores after calibration:
Style accuracy:       0.5644525289535522
Meaning preservation: 0.8554770350456238
Joint fluency:        0.9154849648475647
Joint score:          0.422428697347641
```